## 1. 资产收益预估统计模型的总体构造




使用机器学习模型进行资产回报的预估，建立如下的加法模型
$$
r_{i, t+1} = E_t(r_{i, t+1}) + \epsilon_{i, t+1} \tag{1}
$$
其中$$
E_t(r_{i, t+1}) = g^*(z_{i, t+1}) \tag{2}
$$

$i = 1, 2, ... N_t$为股票id， $t = 1, 2,...,T$为月份序号。
即第i个股票第t+1期的收益为期望收益 + 误差项。


模型中的期望函数g是因子$z_{i, t+1}$的函数， 模型对不同的股票i和不同的时间t，估计同一个期望函数g，并保证g在样本集外有最大的预测能力。






## 2. 样本集合划分和验证

1. 训练集
2. 验证集
3. 测试集

## 3. 几种主要的机器学习方法

### 3.1 简单线性模型

线性最小二乘法。对每一个因子引入权重$\theta_{i, t}$, $\theta = (\theta_{i, t})$。

$$
g(z_{i, t};\theta) = z^{T}_{i, t} \theta \tag{3}
$$

目标函数：

$$
   L(\theta) = \frac{1}{NT} \sum_1^{N}\sum_1^T (r_{i, t+1} - g(z_{i, t}; \theta))^2
$$

### 3.2 增强Robust的线性模型

带权重的最小二乘:
$$
L(\theta) = \frac{1}{NT} \sum_1^{N}\sum_1^T w_{i, t}(r_{i, t+1} - g(z_{i, t}; \theta))^2  \tag{5}
$$

权重项$w_{i, t}$, 可以反比于某一个月的股票种类数。这样月份t对于损失的贡献是一样的。

也可以正比于证券i在时间t的市值。


使用Huber loss：

$$
L(\theta) = \frac{1}{NT} \sum_1^{N}\sum_1^T H((r_{i, t+1} - g(z_{i, t}; \theta)); \xi) \tag{6}
$$

where:
$$
H(X, \xi) = \begin{cases}
x^2& if  |x| <= \xi\\
 2\xi |x| - \xi^2&   if |x| > \xi
\end{cases}
$$

即Huber Loss在差异值小于$\xi$时，用的是平方误差，大于$\xi$时用的线性误差。HuberLoss针对outlier（特异点）具有较好的稳定性，不会因为特异点的存在使得loss function急剧增大。


![](pics/huber.JPG)


### 3.3 带惩罚项的线性模型(Elastic Net, Lasso, Ridge)

惩罚因子对模型的复杂性进行惩罚，降低过拟合的可能性。

$$
L(\theta; .) = L(\theta) + \phi(\theta)  \tag{7}
$$



Elastic Net惩罚模型：
$$
\phi(\theta, \lambda, \rho) = \lambda (1 - \rho)\sum_{j=1}^{P}|\theta_j|  + \frac{1}{2}\lambda\rho\sum_{j=1}^P\theta_j^2 \tag{8}
$$

Elastic Net惩罚模型是说，综合使用了L1(Lasso)和L2(Ridge)的惩罚方式，当$\rho=0$时，就是Lasso回归， 当$\rho=1$时就是Ridge回归。


### 3.4 降维方法

使用Lasso或Ridge回归的方法，会使得因子的系数$\omega$等于0或者接近于0。但是当因子之间高度耦合时，特征选择(feature selection)反而不会得到最优解。
这时降维方法可以派上用场。降维方法是一种特征加权平均的方法。可以减小特征之间的耦合度，减小噪音。主要有两种降维方法： 主成分回归PCR和偏最小二乘法PLS。

#### 3.4.1 主成分回归PCR

主成分回归主要分成两步： 1. 对原始数据做主成分分析(PCA)，找到最重要的$k$个主成分分量。 2. 对这$k$个主成分，进行普通最小二乘回归。

*PCR的优点：降低特征之间的耦合关系，主成分之间相互正交独立。通过特征的线性组合，减小了随机噪音。降低了维度，降低了运算复杂度。*

*PCR的缺点：在做主成分分析降维时，没有考虑到每个因子对回归结果的重要性。*


以下简述主成分分析PCA的主要原理：


假设我们得到一批数据，有$p$个特征维度，分别为$x_1, x_2, x_3, ...., x_p $，向量表示为$X = (x_1, x_2, ..., x_p)^{'}$ 。设均值向量为$\mu$，协方差矩阵为$\Sigma$。

对$X$进行线性变换，得到新的综合变量，用$Y$表示，
$$
\begin{cases}
Y_1 = \mu_{11} X_1 + \mu_{12}X_2  + ... + \mu_{1p}X_p\\
Y_2 = \mu_{21} X_1 + \mu_{22}X_2  + ... + \mu_{2p}X_p\\
...\\
Y_p = \mu_{p1} X_1 + \mu_{p2}X_2  + ... + \mu_{pp}X_p
\end{cases}
$$

我们希望得到的$Y$方差尽可能大且各$Y_i$之间互相独立。又因为
$$
var(Y_i) = var(\mu_i^{'}X)  = \mu_i^{'}\Sigma\mu_i^{'}
$$
为防止方差无限大，做个限制，限制

(1) $\mu_i$是单位正交向量

(2) $Y_i, Y_j（i!=j, i, j=1,2,3...., p）$不相关。

(3) $Y_1$是方差最大者， $Y_2$是$Y_1$不相关的方差最大者。 依次类推，$Y_p$是与$Y_1, Y_2, ..., Y_{p-1}$都不相关的方差最大者。


由以上原则决定的综合变量$Y_1, Y_2, ... Y_p$称为第1， 第2， 。。。第p个主成分。



#### 3.4.2 偏最小二乘回归PLS






### 3.5 广义线性模型

### 3.6 集成学习方法: Boosting Regression Trees和Random Forests

### 3.6.1 回归树和分类树

非参数模型。寻找特征的之间组合，实现对样本空间的划分。


对于分类树，有三种算法： ID3, C4.5, CART。

对于回归树：CART。


**ID3**:  

输入： 训练数据集D，特征集A, 阈值$\epsilon$

输出：决策树T

（1）若D中所有实例属于同一类$C_k$, 则T为单节点树， 并将类$C_k$作为该节点的类标记，返回T

（2）若$A=\emptyset$, 则T为单节点树，并将D中实例数最大的类$C_k$作为该节点的类标记，返回T

（3）否则，计算A中各个特征对D的信息增益，选择信息增益最大的特征$A_g$

（4）如果$A_g$的信息增益小于阈值$epsilon$， 则T为单节点树，并将D中实例数最大的类$C_k$作为该节点的类标记，返回T

（5）否则，对$A_g$的每一个可能只$a_i$， 依$A_g = a_i$将D分割为若干非空子集$D_i$，将$D_i$中实例最大的类作为标记，构建子节点，由节点及其子节点构成树T，返回T

（6）对第i个子节点，以$D_i$为训练集，以$A - {A_g}$为特征集，递归调用（1）~（5），得到$T_i$，返回$T_i$




**C4.5**:

类似于ID3, 选择特征时，使用信息增益比。




**CART分类**:

输入：训练数据集D
输出：CART分类树

从根节点开始，递归地对每个节点进行以下操作，构建二叉决策树。

（1）设节点的训练数据集D，计算现有特征对该数据集的基尼指数。此时，对每一个特征A，对其可能的每一个值$a$，根据样本点对$A=a$的测试为“是”或“否”将D分割成$D_1, D_2$两部分，计算$A=a$时的基尼指数。


（2）在所有的特征A以及它们所有可能的切分点$a$中，选择基尼指数最小的特征及其对应的切分点作为最优特征与最优切分点。依最优特征与最优切分点，从现节点生成两个子节点，将训练数据集依特征分配到两个子节点中去。

（3）对两个子节点递归调用(1),(2)，直至满足停止条件。

（4）生成CART决策树。

算法停止的条件是节点中的样本个数小于阈值，或者样本集的基尼指数小于预订阈值（样本基本属于同一类），或者没有更多特征。




**CART回归**：

输入： 训练数据集D

输出：回归树$f(x)$

在输入数据空间上，递归地将每个区域划分为两个子区域并决定每个子区域上的输出值，构建二叉决策树：

（1） 选择最优切分变量j与切分点s，求解

$$
min_{j,s} \left[min_{c_1} \sum_{x_i \in R_1(j,s)} (y_i - c_i) ^2  + min_{c_2} \sum_{x_i \in R_2(j, s)} (y_i - c_2)^2 \right]
$$

遍历j，扫描s，使得上式最小。

（2） 用选定的(j, s)划分区域并决定输出值:
$$
R_1(j, s) = \left\{ x | x^j <= s \right\}, R_2(j, s) = \left\{x | x^{j} > s\right\}
$$


$$
c_m = \frac{1}{N_m} \sum_{x_i\in R_m(j, s)} y_i, x\in R_m, m = 1, 2
$$


（3） 继续对两颗子树调用(1), (2)，直到满足停止条件。

（4） 划分的m个子区域$R_1, R_2, ...., R_m$， 生成决策树：
    $$
        f(x) = \sum_{m=1}^{M} c_mI(x\in R_m)
    $$


### 3.6.2 Boosting

Boosting框架： 多个较弱的分类器进行组合，加法模型。本次迭代的tree模型结果加到之前的树模型中，使得残差最小化。

$$
g(z) = \sum_{i=1}^{k} f_i(z)
$$


### 3.6.2.1 GBDT

Gradient Based Decision Tree。 在GBDT中，使用的树都是cart回归树。
例如有若干个样本点$(x_1, y_1), (x_2, y_2).....(x_n, y_n)$。 希望构造一个拟合函数F(x)，使得经验风险Loss(y, F(x))最小化。

如果我们已经学习到一个拟合函数$f_1(x)$, 这时我们得到了每个样本点的拟合结果: $$(x_1, f_1(x_1)), (x_2, f_1(x_2)), ...., (x_n, f_1(x_n))$$, 与此同时可以得到每个样本点的残差$$r_i = y_i - f_1(x_i)$$, 这时我们希望学习一个新的函数f_2(x), 它可以最优拟合这个残差。这是一个新的拟合问题，即boosting算法的第二次迭代。得到$f_2(x)$。为避免过拟合，引入一个学习步长$\lambda$, 第二次残差为 $$ y_i - f_1(x_i) - \lambda f_2(x_i)$$


如此迭代直到迭代的最大步骤数。


这里我们可以假设每次拟合的损失函数是平方差损失函数(MSE)，$$ loss_k = (y_i - f_k(x_i))^2$$, 这个损失函数的一阶导就是上面提到的残差。拟合残差就是拟合损失函数的梯度。使用不同的损失函数就拟合不同的梯度，故名Gradient Based Decision Tree。在实际中，MSE的损失函数也不是最好的，因为对Outlier点比较敏感，这时可以改用Huber Loss。迭代拟合时只要求损失函数梯度即可。

$$ f_t(x) \approx \frac{\partial loss}{\partial x}$$

在Boosting框架中，可以选取大量的树(如>100颗), 但是每棵树非常简单，这样可以有效防止过拟合。具体操作为：

$$ \hat{y}_i^0 = 0 $$
$$ \hat{y}_i^1 = \hat{y}_i^0 + \lambda f_1(x_i) $$
$$ \hat{y}_i^2 = \hat{y}_i^1 + \lambda f_2(x_i) $$
$$ \hat{y}_i^3 = \hat{y}_i^2 + \lambda f_3(x_i) $$


$$...$$
$$ \hat{y}_i^{t} = \hat{y}_i^{t-1} + \lambda f_t(x_i) $$



scikit-learn软件包：

sklearn.ensemble.GradientBoostingClassifier(https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html)

sklearn.ensemble.GradientBoostingRegressor(https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingRegressor.html)


在实现中，GBDT中每一颗树是在前面的树构建完成后开始构建的，是顺序执行的。每一颗树的每次节点分裂也是顺序的使用贪心算法遍历每个可能的特征和分裂值。算法没有并行操作。


### 3.6.2.2 Xgboost

在GBDT中， 每次迭代是对损失函数的梯度进行估计拟合，而在Xgboost中，是对损失函数进行二阶泰勒展开逼近。

$$ f(x + \Delta (x)) = f(x) + f'(x)\Delta x + \frac{1}{2} f''(x) \Delta x^2 $$

故而：
$$obj^t = \sum_{i=1}^{N} [l(y_i, \hat{y_i^{t-1}}) + g_i f_t(x_i) + h_i f_t^2(x_i)]$$

其中 $ g_{i}$ 为损失函数的一阶导，$ h_{i} $ 为损失函数的二阶导。


xgboost软件包：
https://xgboost.readthedocs.io/en/release_0.72/index.html

xgboost工具支持自定义代价函数，只要函数可一阶和二阶求导。
xgboost在代价函数里加入了正则项，用于控制模型的复杂度。正则项里包含了树的叶子节点个数、每个叶子节点上输出的score的L2模的平方和。从Bias-variance tradeoff角度来讲，正则项降低了模型variance，使学习出来的模型更加简单，防止过拟合，这也是xgboost优于传统GBDT的一个特性。

在实现中，xgboost的并行不是tree粒度的并行， xgboost在每一次迭代中依然是顺序构建每一颗树。但是在单棵树中广泛使用了C++ 的openmp API,使得在特征split, 最优分割值的遍历上是多线程并行的。





###  3.7 神经网络

#### Hidden Layers

前向传播的隐含层由全连接的神经元$w_i$和偏置项$b_i$及激活函数$\phi(x)$表示。

$$
z_{i+1} = f_{i+1}(z_i) = \phi(w_{i+1}*z_i + b_{i+1})
$$

其中$f_{i+1}$表示第i+1层网络， $z_i$表示第i层的输出。

#### Activation Function

$$
sigmoid(x) = \frac{1}{1 + e^{-x}}
$$


$$
ReLu(x) = \begin{cases}
0\   if x < 0 \\
x\ otherwise
\end{cases}
$$


导函数：

$$
\frac{d sigmoid(x)}{dx} = sigmoid(x) (1 - sigmoid(x))
$$
当 $x -> +\infty, or x -> -\infty$, 导函数趋于0： 梯度消失。

$$
\frac{d ReLu(x)}{dx} = \begin{cases}
0 \  if x<0 \\
1 \   otherwise
\end{cases}
$$
没有梯度消失。



#### Optimizers

Stochatic Gradient Descent, Momentum, Adagrad, RMSProp, Adam。


#### Batch Normalization
在网络的每一层输入值进行“0， 1”标准化，使得网络的输入均值近似为0， 方差近似为1。目的是使得网络的输入值能够更好的落入激活函数的敏感区间，避免梯度消失和过拟合。标准化时，没有严格的执行均值0，方差1， 而是给了一个小的偏差$\beta$和$\gamma$, 这两个参数分别接近0和1，作为参数由神经网络进行学习。

